In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# To Do: Figure out Vle joins

#studentVle = pd.read_csv('dataset/studentVle.csv')
#vle = pd.read_csv('dataset/vle.csv')

In [3]:
def create_student_df():
    # Read in .csv's to be joined
    studentInfo = pd.read_csv('dataset/studentInfo.csv')
    studentRegistration = pd.read_csv('dataset/studentRegistration.csv')

    # Drop any unnecessary columns (can be edited for feature extraction)
    si_drop_cols = ['gender', 'region', 'highest_education', 'imd_band', 'disability']
    si = studentInfo.drop(columns=si_drop_cols)

    # Merge, inner join
    student_df = pd.merge(si, studentRegistration, how='inner', on=['code_module', 'code_presentation', 'id_student'])

    return student_df


In [4]:
def assessment_namer(id_assessment):
    Assess_1s = [1752, 1758, 14984, 14996, 15008, 15020, 24282, 24291, 25334, 25348, 25355, 25362, 30709, 30714, 30719, 34860, 34873, 34886, 34899]
    Assess_2s = [1753, 1759, 14985, 14997, 15009, 15021, 24283, 24292, 25335, 25349, 25356, 25363, 30710, 30715, 30720, 34861, 34874, 34887, 34900]

    if id_assessment in Assess_1s:
        return "A1"
    elif id_assessment in Assess_2s:
        return "A2"
    else:
        return "Drop"


In [5]:
def create_assessment_df():
    # Read in .csv's to be joined
    assessments = pd.read_csv('dataset/assessments.csv')
    studentAssessment = pd.read_csv('dataset/studentAssessment.csv')

    # Drop any unnecessary columns
    sa_drop_cols = ['is_banked']
    sa = studentAssessment.drop(columns=sa_drop_cols)

    # Merge, left join
    assessment_df = pd.merge(sa, assessments, how='left', on='id_assessment')

    # Dropping 'assessment_type' == 'CMA' due to extreme course differences
    # Also dropping the final exam because we want to catch students BEFORE then
    a_filtered = assessment_df[assessment_df.assessment_type == 'TMA']

    # Mapping with assessment_namer
    a_filtered['assessment_name'] = a_filtered['id_assessment'].apply(assessment_namer)
    a_filtered = a_filtered[a_filtered.assessment_name != 'Drop']
    

    return a_filtered



In [6]:
def wide_form_sa():
    sa = create_assessment_df()

    # Combine date submitted and date into -/+
    sa['days_from_due'] = sa['date_submitted'] - sa['date']

    # Drop Assessment Type, Weight
    sa.drop(columns=['assessment_type', 'weight', 'date_submitted', 'date', 'id_assessment'], inplace=True)

    inds = ['id_student', 'code_module', 'code_presentation']
    vals = ['score', 'days_from_due']

    sa_wide = sa.pivot_table(
                values = vals,
                columns = 'assessment_name',
                index = inds
    )

    
    sa_wide.columns = ["_".join(a) for a in sa_wide.columns.to_flat_index()]
    sa_wide.reset_index(inplace=True)

    return sa_wide


In [7]:
def create_si_sa_df():
    si = create_student_df()
    sa = wide_form_sa()

    # Merge
    si_sa_df = pd.merge(si, sa, how='inner', on=['code_module', 'code_presentation', 'id_student'])

    return si_sa_df

In [8]:
create_si_sa_df()

,code_module,code_presentation,id_student,age_band,num_of_prev_attempts,studied_credits,final_result,date_registration,date_unregistration,days_from_due_A1,days_from_due_A2,score_A1,score_A2
0,AAA,2013J,11391,55<=,0,240,Pass,-159.0,NaN,-1.0,-1.0,78.0,85.0
1,AAA,2013J,28400,35-55,0,60,Pass,-53.0,NaN,3.0,-2.0,70.0,68.0
2,AAA,2013J,31604,35-55,0,60,Pass,-52.0,NaN,-2.0,-3.0,72.0,71.0
3,AAA,2013J,32885,0-35,0,60,Pass,-176.0,NaN,7.0,21.0,69.0,30.0
4,AAA,2013J,38053,35-55,0,60,Pass,-110.0,NaN,0.0,10.0,79.0,69.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22629,FFF,2014J,2684048,0-35,0,120,Withdrawn,-16.0,12.0,14.0,NaN,68.0,NaN
22630,FFF,2014J,2686035,0-35,0,135,Withdrawn,-30.0,236.0,-1.0,1.0,90.0,72.0
22631,FFF,2014J,2693932,0-35,1,120,Pass,-72.0,NaN,-3.0,-8.0,90.0,88.0
22632,FFF,2014J,2694139,35-55,0,60,Withdrawn,-81.0,137.0,0.0,24.0,66.0,68.0
